In [1]:
# Transmitter.py 

import numpy as np
from matplotlib import pyplot as plt
from PIL import Image  # Import PIL for image handling

def rrc_filter(beta, sps, num_taps):
    """Generates a Root Raised Cosine (RRC) filter."""
    t = np.arange(-num_taps//2, num_taps//2 + 1) / sps
    pi_t = np.pi * t
    four_beta_t = 4 * beta * t

    with np.errstate(divide='ignore', invalid='ignore'):
        numerator = np.sin(pi_t * (1 - beta)) + 4 * beta * t * np.cos(pi_t * (1 + beta))
        denominator = pi_t * (1 - (four_beta_t) ** 2)
        h = numerator / denominator

    # Handle t = 0 case
    h[np.isnan(h)] = 1.0 - beta + (4 * beta / np.pi)
    # Handle t = ±1/(4β)
    t_special = np.abs(t) == (1 / (4 * beta))
    h[t_special] = (beta / np.sqrt(2)) * (
        ((1 + 2 / np.pi) * np.sin(np.pi / (4 * beta))) +
        ((1 - 2 / np.pi) * np.cos(np.pi / (4 * beta)))
    )

    # Normalize the filter
    h /= np.sqrt(np.sum(h**2))

    return h

def simulate_transmission(signal, snr_db):
    """Simulates transmission over an AWGN channel."""
    # Calculate signal power and noise power
    signal_power = np.mean(np.abs(signal)**2)
    snr_linear = 10**(snr_db / 10)
    noise_power = signal_power / snr_linear

    # Generate white Gaussian noise
    noise = np.sqrt(noise_power / 2) * (np.random.randn(len(signal)) + 1j * np.random.randn(len(signal)))

    # Add noise to the signal
    received_signal = signal + noise

    return received_signal

In [9]:
# Configuration parameters
samples_per_symbol = 8  # Samples per symbol
beta = 0.2  # RRC filter roll-off factor
num_taps = 101  # Number of RRC filter taps
snr_db = 30  # Signal-to-noise ratio in dB

# Generate preamble
# Create a random sequence for half the preamble
half_preamble_bits = np.random.randint(0, 2, size=10)  # 100 bits
# Repeat the sequence to form the full preamble
preamble_bits = np.concatenate((half_preamble_bits, half_preamble_bits))
# Map bits to BPSK symbols
preamble_symbols = 2 * preamble_bits - 1

# Load image
image = Image.open('telecomunicaciones-2.webp')  # Replace with your image path
image = image.convert('RGB')
width, height = image.size
image_array = np.array(image)
image_flat = image_array.flatten()
image_bits = np.unpackbits(image_flat)

# Create header with image size (16 bits for width and height)
width_bytes = np.array([width], dtype='>u2').view(np.uint8)
height_bytes = np.array([height], dtype='>u2').view(np.uint8)
width_bits = np.unpackbits(width_bytes)
height_bits = np.unpackbits(height_bytes)
header_bits = np.concatenate((width_bits, height_bits))

# Concatenate preamble, header, and image bits
bits = np.concatenate((preamble_bits, header_bits, image_bits))

# Map bits to BPSK symbols (-1, +1)
symbols = 2 * bits - 1

# Upsample symbols
symbols_upsampled = np.zeros(len(symbols) * samples_per_symbol)
symbols_upsampled[::samples_per_symbol] = symbols

# Generate RRC filter
rrc_coef = rrc_filter(beta, samples_per_symbol, num_taps)

# Filter the signal
signal_filtered = np.convolve(symbols_upsampled, rrc_coef, mode='same')

# Normalize signal amplitude
signal_filtered /= np.max(np.abs(signal_filtered))

# Convert to complex baseband signal
signal_complex = signal_filtered.astype(np.complex64)

# Simulate transmission over an AWGN channel
received_signal = simulate_transmission(signal_complex, snr_db)

print("Transmission simulation complete. Signal saved for receiver processing.")

Transmission simulation complete. Signal saved for receiver processing.


In [3]:
def schmidl_cox_algorithm_vectorized(signal, L, threshold=0.8):
    N = len(signal)
    # Compute P(d) using convolution
    r_conj = np.conj(signal)
    P = np.correlate(signal[L:], r_conj[:-L], mode='valid')
    # Compute R(d) using sliding window sum of squares
    abs_sq = np.abs(signal)**2
    R = np.convolve(abs_sq, np.ones(2*L), mode='valid')
    # Compute M(d)
    M = np.abs(P)**2 / R**2
    # Normalize and detect peaks
    M = M / np.max(M)
    peaks = np.where(M > threshold)[0]
    if len(peaks) > 0:
        d_max = peaks[0]
        return d_max, M
    else:
        return None, M


In [10]:
# Configuration parameters
samples_per_symbol = 8  # Samples per symbol
beta = 0.2  # RRC filter roll-off factor
num_taps = 101  # Number of RRC filter taps

# Detect preamble using Schmidl-Cox algorithm
# Set L to the length of half the preamble in samples
L = len(half_preamble_bits) * samples_per_symbol
print(f"Length of half the preamble in samples: {L}")
d_max, M = schmidl_cox_algorithm_vectorized(received_signal, L)

if d_max is not None:
    print(f"Preamble detected at index: {d_max}")

    # Matched filtering with RRC filter
    rrc_coef = rrc_filter(beta, samples_per_symbol, num_taps)
    matched_filtered_signal = np.convolve(received_signal, rrc_coef, mode='same')

    # Extract the portion of the signal containing the data
    start_idx = d_max + 2 * L
    data_signal = matched_filtered_signal[start_idx:]

    # Sample symbols
    symbol_indices = np.arange(0, len(data_signal), samples_per_symbol)
    sampled_symbols = data_signal[symbol_indices]

    # Decode symbols to bits
    received_bits = (np.real(sampled_symbols) > 0).astype(np.uint8)

    # Extract header bits (32 bits)
    header_bits = received_bits[:32]

    # Convert header bits to width and height
    width_bits = header_bits[:16]
    height_bits = header_bits[16:32]

    width_bytes = np.packbits(width_bits)
    height_bytes = np.packbits(height_bits)

    # Convert bytes to integers (ensure they are standard Python ints)
    width = int(np.frombuffer(width_bytes.tobytes(), dtype='>u2')[0])
    height = int(np.frombuffer(height_bytes.tobytes(), dtype='>u2')[0])

    print(f"Image Width: {width}, Image Height: {height}")

    # Calculate the number of bits for the image
    num_image_bits = width * height * 3 * 8  # 3 colors (RGB), 8 bits per color

    # Extract image bits
    image_bits = received_bits[32:32 + num_image_bits]

    if len(image_bits) < num_image_bits:
        print("Not enough bits received to reconstruct the image.")
    else:
        # Convert bits to bytes
        image_bytes = np.packbits(image_bits)

        # Convert bytes to image array
        image_array = np.frombuffer(image_bytes, dtype=np.uint8)

        # Ensure the total number of elements matches
        expected_size = width * height * 3
        if image_array.size != expected_size:
            print(f"Expected image array size: {expected_size}, but got: {image_array.size}")
            print("Cannot reshape the array due to size mismatch.")
        else:
            image_array = image_array.reshape((height, width, 3))

            # Create and display the image
            image = Image.fromarray(image_array, 'RGB')
            image.show()

else:
    print("Preamble not detected in the received signal.")

Length of half the preamble in samples: 80
Preamble detected at index: 0
Image Width: 1024, Image Height: 597
